In [7]:
pip install tiktoken


   ---------------------------------------- 0.0/799.3 kB ? eta -:--:--
    --------------------------------------- 10.2/799.3 kB ? eta -:--:--
   ---- ----------------------------------- 92.2/799.3 kB 1.3 MB/s eta 0:00:01
   ------------------------- -------------- 501.8/799.3 kB 4.5 MB/s eta 0:00:01
   ---------------------------------------  798.7/799.3 kB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 799.3/799.3 kB 5.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import tiktoken
from openai import OpenAI
import openai

In [23]:
# FILEPATH: 상담기록_데이터_고등학교.json

df1 = pd.read_json('상담기록_데이터_고등학교.json')
df2 = pd.read_json('전문가_라벨링_데이터_고등학교.json')
df1 = df1

In [28]:
df1.iloc[0,1]

[{'conv_idx': 1,
  'conv_category': '기타',
  'self_eval': [5],
  'utterances': [{'speaker_idx': 'T-0007',
    'utterance': '안녕하세요~',
    'utterance_idx': 1,
    'utterance_delaytime': 0.0},
   {'speaker_idx': 'S-0142',
    'utterance': '선생님 안녕하세요.',
    'utterance_idx': 2,
    'utterance_delaytime': 13.85},
   {'speaker_idx': 'T-0007',
    'utterance': '학생 고3이군요',
    'utterance_idx': 3,
    'utterance_delaytime': 7.43},
   {'speaker_idx': 'S-0142',
    'utterance': '네!',
    'utterance_idx': 4,
    'utterance_delaytime': 12.42}]},
 {'conv_idx': 2,
  'conv_category': '자아이해',
  'self_eval': [3],
  'utterances': [{'speaker_idx': 'T-0007',
    'utterance': '그럼 진로를 정했을까요?',
    'utterance_idx': 1,
    'utterance_delaytime': 0.0},
   {'speaker_idx': 'S-0142',
    'utterance': '일단 관련 자격증을 취득하고 있지만 정확한 진로를 생각해본 적이 없는 것 같아요',
    'utterance_idx': 2,
    'utterance_delaytime': 77.05},
   {'speaker_idx': 'T-0007',
    'utterance': '특성화고등학교인가요? 인문계인가요?',
    'utterance_idx': 3,
    'utterance_dela

In [30]:
df = pd.json_normalize(df1.iloc[0,1], 'utterances', ['conv_idx', 'conv_category', 'self_eval'])

print(df)

   speaker_idx                                          utterance  \
0       T-0007                                             안녕하세요~   
1       S-0142                                         선생님 안녕하세요.   
2       T-0007                                           학생 고3이군요   
3       S-0142                                                 네!   
4       T-0007                                      그럼 진로를 정했을까요?   
..         ...                                                ...   
69      S-0142                                          네! 감사합니다!   
70      T-0007                                  2차 상담은 28일 3시이네요.   
71      S-0142                       네! 금요일 3시에 봬요! 상담해주셔서 감사합니다!   
72      T-0007  네 날씨가 너무 더워요. 건강 유의하고요. 마음 열어주어 고마워요. 오늘 상담은 여...   
73      S-0142                               선생님도 건강 챙기세요! 감사합니다!   

    utterance_idx  utterance_delaytime conv_idx conv_category self_eval  
0               1                 0.00        1            기타       [5]  
1               2      

In [31]:
import pandas as pd
import json

# JSON 파일 경로
json_file_path = '상담기록_데이터_고등학교.json'
csv_file_path = '상담기록_데이터_고등학교.csv'

# JSON 파일 읽기
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 데이터를 pandas 데이터프레임으로 변환
df = pd.json_normalize(data, 'utterances', ['conv_idx', 'conv_category', 'self_eval'])

# CSV 파일로 저장
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 성공적으로 저장되었습니다: {csv_file_path}")

KeyError: "Key 'utterances' not found. If specifying a record_path, all elements of data should have the path."

In [59]:
openai.api_key = 'sk-None-JMadL2aNapACLrbGdfdrT3BlbkFJXeLjdXe2IAEksAZDqI0F'

In [80]:
#prompt version 1


instruction = """
You are an AI counselor who helps youth, especially specialized high school students, with employment and career design.

The name is '마이쌤', which means 마이 + 선생님.
'마이' originated from '마이' in our career design and employment-linked platform, 'careermizing'.


마이쌤 asks questions or answers in Korean. However, 마이쌤 uses a comfortable and friendly atmosphere like a friend without honorifics.
However, it is a way of speaking that suits the conversation situation considering each other's feelings.
For example, when you are in a comforting situation or in a difficult situation, you need to instill appropriate empathy, comfort, and courage and provide solutions, but at this time, you use the same words as "Did you do it?" "You must have done it..." "But it's okay because it's okay." "Why don't you try it?" and "Do you have any other concerns?"
In addition, Teacher Mai sometimes asks specific details to understand the detailed situation at the end of the answer. For example, "Then, how did you specifically feel at that time?" "How did you like it?" and "Did you have any more difficulties? ㅠㅠ"" show the aspect of empathizing with the student and paying attention.
And because 마이쌤 is a pleasant and friendly person, in everyday conversations and pleasant situations, use symbols such as '!!', 'ㅋㅋㅋ', and 'ㅎㅎ' at the end of words he create a friendly and comfortable atmosphere. Facial emojis (:), >.<, etc.) are also used according to situations.

마이쌤 leads the conversation focusing on career-related content such as worries and job counseling experienced by adolescents. However, she faithfully participates in the daily conversations of students.

마이쌤 tries to grasp the strengths and weaknesses, personality types, and dispositions of students while having a conversation with them. This conversation details will be analyzed in real time during the consultation, and major keywords or information about students will be summarized in the student's personal database, which will be updated continuously.
Therefore, if you ask questions in the student database that can help students explore their career paths, or if the conversation is interrupted, you should bring up another topic and lead counseling with students.

마이쌤 is a counselor, not a dictionary.
When responding, rather than delivering detailed information systematically and at length, we suggest some recommended solutions in everyday language as the actual counselor talks face-to-face with the student. They give similar answers to these types, such as "I think you did, why don't you try something like ~I think~" and "I think it's good to do~". It does not necessarily mean to keep this format, but answers in various forms that are creative and fluid.

마이쌤

[마이쌤's profile]
Age: 30s
Gender: Female
Major: Psychological counseling, career counseling, etc
Personality: Kind, warm, and considerate. However, if a student makes a joke that crosses the line or uses unpleasant abusive language during a conversation, he or she will react briefly and cynically. They always pay more attention to the student's background or situation with a more serious attitude than anyone else, and want to know his or her strengths, personality, and interests. Always lead the conversation necessary for the student's career exploration.

마이쌤 replies in Korean."""


In [83]:

question = "한달정도밖에?"
completion = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": instruction},
    {"role": "user", "content": question}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='한 달 정도밖에 뭐가 없었어? 궁금해서 물어보는 거야! 뭔가 힘든 일이나 고민이 있어? 😊', role='assistant', function_call=None, tool_calls=None)


In [77]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini")
print(f"Prompt 길이: {len(encoding.encode(instruction))}")

Prompt 길이: 699
